In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")

text = text[:100000]


Taille du vocabulaire : 6745


In [2]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(5, len(sequence_data)):
    words = sequence_data[i-5:i+1]
    sequences.append(words)

sequences = np.array(sequences)




In [3]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  
y = sequences[:, -1]  
y = to_categorical(y, num_classes=vocab_size)





In [4]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(vocab_size, 40))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(120)))
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax'))
 

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=80, verbose=1)



Epoch 1/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0315 - loss: 7.0830
Epoch 2/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0432 - loss: 6.0764
Epoch 3/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0659 - loss: 5.8115
Epoch 4/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0818 - loss: 5.5884
Epoch 5/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0948 - loss: 5.3508
Epoch 6/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1078 - loss: 5.1344
Epoch 7/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1226 - loss: 4.9367
Epoch 8/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1327 - loss: 4.7347
Epoch 9/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1362 - loss: 4.5656
Epoch 10/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1558 - loss: 4.3572
Epoch 11/80
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.1739 - loss: 4.1516
Epoch 12/80
568/568 ━━━━━━━━━━━━━━━━━━━━

In [6]:
input_text = "you have a bad financial"
predict_next_words = 45


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-5:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

you have a bad financial astonishment and the party was not joined in a day in the country but you must acknowledge to be true certainly my dear nobody said there were but as to be meeting with many fortune eyes in the room he was lively and unreserved danced
